# Predicting FMC with trained 10h Model

Model trained on hourly dead 10h. Applying model to local weather and comparing to field observations.

Geographic info from Mesonet slapout: https://www.mesonet.org/about/station-information?stid=slap

## Setup

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from src.models import moisture_rnn as mrnn
from src.utils import read_yml

In [ ]:
# Read Trained model
params = read_yml("models/params.yaml")
rnn = mrnn.RNN_Flexible(params=params)

In [ ]:
rnn.load_weights('models/rnn.keras')

## Handle observed Carlson Data

In [ ]:
weather = pd.read_excel("data/processed_data/ok_weather.xlsx")
fm10 = pd.read_excel("data/processed_data/ok_data_10h.xlsx")

In [ ]:
# Calculate Variables
rh = weather.rh
temp = weather.temp
Ed = 0.924 * rh**0.679 + 0.000499 * np.exp(0.1 * rh) + 0.18 * (21.1 + 273.15 - temp) * (1 - np.exp(-0.115 * rh))
Ew = 0.618 * rh**0.753 + 0.000454 * np.exp(0.1 * rh) + 0.18 * (21.1 + 273.15 - temp) * (1 - np.exp(-0.115 * rh))

hod = weather.date.dt.hour
doy = weather.date.dt.dayofyear

# Geographic Variables from Slapout station

In [ ]:
X = pd.DataFrame({
    "Ed": Ed,
    "Ew": Ew,
    "solar": weather["k.down"],
    "wind": weather["wind.speed"],
    "elev": 774,
    "lon": -100.261920,
    "lat": 36.597490,
    "rain": weather.precip,
    "hod": hod,
    "doy": doy
})

print(f"Columns match: {np.all(X.columns == params['features_list'])}")

In [ ]:
# Get scaler from RNN Data


In [ ]:
#
XX = X.to_numpy().reshape(1, *X.shape)
preds = rnn.predict(XX).flatten()